In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from Map import Map
from Tabu import Tabu
from TwoStage import TwoStage
from L_shaped import MasterProblem as mp
import matplotlib.pyplot as plt
import matplotlib.pylab as pl 
import matplotlib.cm as cm
import time
import networkx as nx
import matplotlib as mpl
from gurobipy import *
import csv

**Modelling Parameters**

In [3]:
rides = 5
bus = 2
scenarios = 20
MIPGap = 0.01
TimeLimit = 18000
probability = [0.7, 0.1, 0.15, 0.05]

**Defining the Map**

In [4]:
mappy = Map(rides, seed=200)

**Plotting Functions**

In [5]:
def plot_trend(ub,lb,base):
    x = list(range(len(ub)))
    figure = plt.figure()
    plt.plot(x,ub,label='upper-bound')
    plt.plot(x,[base]*len(ub),label='two-stage')
    plt.plot(x,lb,label='lower-bound')
    plt.annotate(int(ub[-1]),[x[-1],ub[-1]])
    plt.legend()
    plt.savefig('./figures/trend.png')
    plt.show()

colors = ['green','blue','yellow','red','pink'] 
def displaygraph(n, e, modname):
    for k in e.keys():
        for i, j in e[k]:
            plt.plot((n[i][0],n[j][0]),(n[i][1],n[j][1]), color=colors[k] ,marker='o', linewidth=2, linestyle='dashed')
    for i in n:
        plt.annotate(i,n[i],textcoords="offset points",xytext=(0,10),ha='center')
    plt.savefig('./figures/'+modname+'.png')
    plt.show()


#### Model formulation

In [6]:
def Mod1(aux=False):
    t1 = time.time()
    drpstw = mp(mappy,bus=bus,scenarios=scenarios, probability=probability)
    drpstw.initialize()
    drpstw.setMIP()
    drpstw.variables.th.obj=0
    if aux:
        drpstw.setLowerBound()
        for s in range(drpstw.scenarios):
            drpstw.submodel[s].model.params.MIPGap = MIPGap
#             drpstw.submodel[s].model.params.OutputFlag = 1
    else:
        drpstw.model.params.TimeLimit = TimeLimit
        drpstw.model.optimize()
    t1 = time.time() - t1
    return drpstw, t1

In [7]:
def Mod2(model=None):
    t2 = time.time()
    twosp = TwoStage(mappy, bus=bus, scenarios=scenarios, probability=probability)
    twosp.model.params.TimeLimit = TimeLimit
    if model is not None:
        for i,j,k in twosp.MP.variables.x.keys():
            twosp.MP.variables.x[i,j,k].start = model.variables.x[i,j,k].X
    twosp.optimize()
    t2 = time.time() - t2
    return twosp, t2

In [8]:
def Mod3():
    t3 = time.time()
    lshaped = mp(mappy, bus=bus, scenarios=scenarios, probability=probability)
    lshaped.initialize()
    lshaped.model.params.TimeLimit = TimeLimit
    lshaped.optimize(skip_init=False)
    t3 = time.time() - t3
    return lshaped, t3

In [9]:
def Mod4(lshaped):
    t4 = time.time()
    if lshaped.submodel is not None:
        tabu = Tabu(lshaped,tabu_iterations=200, tabu_status=3, rtoptm=5, subset=20, tsp=False)
        tabu.tsp = True
    else:
        tabu = Tabu(lshaped,tabu_iterations=200, tabu_status=4, rtoptm=5, subset=20, tsp=True)
    tabu.tabuheuristic()
    t4 = time.time() - t4
    return tabu, t4

In [10]:
def Mod5():
    t3 = time.time()
    lshaped = mp(mappy, bus=bus, scenarios=scenarios, probability=probability)
    lshaped.initialize()
    lshaped.model.params.TimeLimit = TimeLimit
    lshaped.optimize(skip_init=True)
    t3 = time.time() - t3
    return lshaped, t3

### Test Block

In [ ]:
TimeLimit = 18000
tests = [(3,2),(4,2),(5,2),(6,2),(7,2),(9,3),(10,3),(11,3),(13,4),(15,4),(18,5),(20,5),(22,6),(24,6),(25,7),(27,8)]
scenarioslist = [10,20,30]

csv_file = open('./Reports/Report.csv', 'w', newline='')
write = csv.writer(csv_file, delimiter=',')
write.writerow(('Rides','Bus','Scenarios','TSPObj','TSPGap','TSPTime','LshapeObj','LshapeSave','LshapeGap','LshapeTime','TabuObj','TabuSave','TabuTime'))
csv_file.close()
for s in scenarioslist:
    scenarios = s
    for test in tests:
        
        rides, bus = test
        mappy = Map(rides)

        #Two-Stage
        try:    
            twostage, t1 = Mod2()
            if twostage.model.status in [2,9]:
                twostageobj = twostage.model.ObjVal
                twostagegap = twostage.model.MIPGap
                twostagetime = t1
            else:
                twostageobj = 'NA'
                twostagegap = 'NA'
                twostagetime = t1
        except (GurobiError, KeyboardInterrupt):
            twostageobj = 'Failed'
            twostagegap = 'Failed'
            twostagetime = 'Failed'
        #L-shaped
        try:    
            lshaped, t2 = Mod3()
            if lshaped.model.status in [2,9]:
                lshapedobj = lshaped.model.ObjVal
                lshapedsave = lshaped.variables.th.X
                lshapedgap = lshaped.model.MIPGap
                lshapedtime = t2
            else:
                lshapedobj = 'NA'
                lshapedgap = 'NA'
                lshapedtime = t2
                lshapedsave = 'NA'
        except (GurobiError, KeyboardInterrupt):
            lshapedobj = 'Failed'
            lshapedsave = 'Failed'
            lshapedgap = 'Failed'
            lshapedtime = 'Failed'
        #Tabu
        try:    
            tabu, t3 = Mod4(lshaped)
            tabuobj = tabu.best[-1]
            tabusave = tabu.best[-3]
            tabutime = t3
        except (GurobiError, KeyboardInterrupt):
            tabuobj = 'Failed'
            tabusave = 'Failed'
            tabutime = 'Failed'

        csv_file = open('./Reports/Report.csv', 'a', newline='')
        write = csv.writer(csv_file, delimiter=',')
        write.writerow((str(rides),
                        str(bus),
                        str(scenarios),
                        str(twostageobj),
                        str(twostagegap),
                        str(twostagetime),
                        str(lshapedobj),
                        str(lshapedsave),
                        str(lshapedgap),
                        str(lshapedtime),
                        str(tabuobj),
                        str(tabusave),
                        str(tabutime)))
        csv_file.close()
        

In [ ]:
MIPGap = 0.001
tests = [(3,1),(4,1),(5,1),(6,2),(7,2),(8,2),(9,2)]
scenarioslist = [10,15,20]
for s in scenarioslist:
    scenarios = s
    for test in tests:
        rides, bus = test
        mappy = Map(rides)        
#         drpstw, t1 = Mod1()
#         objdrpstw = drpstw.model.ObjVal
        twosp, t2 = Mod2()
        objtwosp = twosp.model.ObjVal
        lshaped, t3 = Mod3() 
        objlshaped = lshaped.model.ObjVal
#         tabu, t4 = Mod4()
#         objtabu = tabu.best[-2]
        write = [
#             str(objdrpstw), str(t1),
            str(objtwosp), str(t2),
            str(objlshaped), str(t3),
#             str(objtabu), str(t4),
            str(scenarios),str(bus),str(rides)]
        with open('report.txt', 'a') as file:
            file.write('\n')
            for w in write:
                file.write(w)
                file.write(',')

### DARP-STW Model

#### Model Solution 

In [ ]:
drpstw, t1 = Mod1()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(drpstw.bus)}
for i, j in drpstw.parameters.edges:
    for k in range(drpstw.bus):
        if drpstw.variables.x[i,j,k].X > 0.5:
            if j != drpstw.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))

objdrpstw = drpstw.model.ObjVal
displaygraph(nodes, edges,'DARP-STW')
nodes,edges

### Two-Stage Stochastic Model

#### Model Solution

In [ ]:
twosp, t2 = Mod2()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(twosp.bus)}
for i, j in twosp.parameters.edges:
    for k in range(twosp.bus):
        if twosp.MP.variables.x[i,j,k].X > 0.5:
            if j != twosp.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))

objtwosp = twosp.model.ObjVal
displaygraph(nodes, edges,'Two-Stage')
nodes,edges

### L-Shaped Method

#### Model Solution

In [11]:
lshaped, t3 = Mod3()

Using license file C:\Users\lavkeshg\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter NodefileStart to 0.5
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Presolve to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter MIPFocus to 2
   Prev: 0  Min: 0  Max: 3  Default: 0
Parameter Heuristics unchanged
   Value: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter TimeLimit to 18000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 898 rows, 365 columns and 3628 nonzeros
Model fingerprint: 0x67ef532a
Variable types: 143 continuous, 222 integer (222 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 9e+00]
  Boun

 12502  3364     cutoff   43        93.04139 -915.41953  1084%  15.6  370s
 12665  3378 -573.55656   35   14   93.04139 -914.76452  1083%  15.5  377s
 12773  3461     cutoff   42        93.04139 -914.46204  1083%  15.5  380s
*12819  3457              50      88.7703679 -914.44085  1130%  15.5  381s
 13067  3518 -661.38314   26   22   88.77037 -913.52965  1129%  15.5  387s
 13228  3537     cutoff   36        88.77037 -913.17184  1129%  15.5  394s
 13360  3571   30.25415   45    6   88.77037 -912.73599  1128%  15.5  399s
 13443  3579     cutoff   49        88.77037 -912.66017  1128%  15.4  402s
 13695  3654 -761.93605   24   24   88.77037 -911.77064  1127%  15.4  412s
 13919  3662     cutoff   44        88.77037 -911.49176  1127%  15.3  415s
 14062  3719     cutoff   46        88.77037 -910.96750  1126%  15.3  420s
 14395  3785     cutoff   31        88.77037 -909.29159  1124%  15.3  437s
 14694  3854     cutoff   38        88.77037 -908.25980  1123%  15.3  449s
 15046  3931     cutoff  

 40585  8202     cutoff   42        68.17722 -765.45637  1223%  13.9 1396s
 41114  8238     cutoff   39        68.17722 -764.98213  1222%  13.9 1411s
 41291  8290     cutoff   42        68.17722 -762.96847  1219%  13.9 1419s
 41969  8334 -201.45079   48   12   68.17722 -760.79020  1216%  13.8 1441s
 42310  8349  -72.01494   37    -   68.17722 -760.55393  1216%  13.8 1448s
 42680  8389     cutoff   48        68.17722 -758.44809  1212%  13.8 1459s
 42747  8393 -468.32931   42   14   68.17722 -758.27603  1212%  13.8 1461s
 43010  8402     cutoff   37        68.17722 -758.07689  1212%  13.8 1466s
 43353  8464     cutoff   46        68.17722 -756.70343  1210%  13.8 1480s
 43516  8478 -679.93662   33   15   68.17722 -756.47499  1210%  13.8 1488s
 43688  8487     cutoff   44        68.17722 -756.39753  1209%  13.8 1491s
 44028  8548     cutoff   43        68.17722 -754.75253  1207%  13.8 1502s
 44584  8581     cutoff   48        68.17722 -754.03145  1206%  13.8 1517s
 44688  8567     cutoff  

 78829  9116 infeasible   55        68.17722 -572.37241   940%  12.9 2626s
 79026  9127 -106.14447   52    7   68.17722 -572.13202   939%  12.9 2631s
 79631  9071     cutoff   44        68.17722 -569.19035   935%  12.8 2647s
 80394  9032     cutoff   41        68.17722 -564.61942   928%  12.8 2675s
 80883  9050     cutoff   43        68.17722 -564.26877   928%  12.8 2684s
 81181  9008     cutoff   46        68.17722 -560.63951   922%  12.8 2697s
 81453  9015     cutoff   31        68.17722 -560.28627   922%  12.8 2701s
 81915  8996     cutoff   50        68.17722 -557.04863   917%  12.8 2720s
 82698  8959     cutoff   39        68.17722 -552.84387   911%  12.8 2742s
 83147  8968     cutoff   44        68.17722 -552.24658   910%  12.8 2746s
 83444  8900     cutoff   50        68.17722 -549.16452   905%  12.8 2759s
 83741  8910     cutoff   43        68.17722 -549.00218   905%  12.8 2762s
 84240  8857     cutoff   46        68.17722 -544.81423   899%  12.7 2779s
 85032  8856     cutoff  

In [ ]:
lshaped.LazyInt

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(lshaped.bus)}
for i, j in lshaped.parameters.edges:
    for k in range(lshaped.bus):
        if lshaped.variables.x[i,j,k].X > 0.5:
            if j != lshaped.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))
bounds = lshaped.getcancel()
objlshaped = lshaped.model.ObjVal
displaygraph(nodes, edges,'L-Shaped')
nodes,edges,bounds

In [ ]:
plot_trend(lshaped.upperbounds[2:],lshaped.lowerbounds[2:],0)

In [ ]:
mostcancel = {i: 0 for i in lshaped.parameters.pickup}
for i in lshaped.parameters.pickup:
    for s in range(lshaped.scenarios):
        mostcancel[i] += 2 - lshaped.submodel[s].sim.alpha[i] - lshaped.submodel[s].sim.alpha[i+lshaped.parameters.rides]
mostcancel = sorted(mostcancel.items(), key=lambda x: (x[1]))
mostcancel

In [12]:
lshaped.printsol()

Routing for bus 1
+---------+------+-------------+---------------+
| Pick-up | Drop | Pickup time | Drop-off time |
+---------+------+-------------+---------------+
|    8    |  5   |    12:48    |     13:04     |
+---------+------+-------------+---------------+
|    5    |  10  |    13:04    |     13:32     |
+---------+------+-------------+---------------+
|   10    |  11  |    13:32    |     14:09     |
+---------+------+-------------+---------------+
|    0    |  3   |    12:11    |     12:20     |
+---------+------+-------------+---------------+
|    3    |  8   |    12:20    |     12:48     |
+---------+------+-------------+---------------+
Routing for bus 2
+---------+------+-------------+---------------+
| Pick-up | Drop | Pickup time | Drop-off time |
+---------+------+-------------+---------------+
|    6    |  7   |    9:36     |     10:04     |
+---------+------+-------------+---------------+
|    2    |  6   |    9:20     |     9:36      |
+---------+------+-------------+-

In [ ]:
sum((1/scenarios)*lshaped.submodel[i].model.ObjVal for i in lshaped.submodel.keys())

### Tabu-Heuristic

#### Model Solution

In [ ]:
tabu, t4 = Mod4(lshaped)

In [ ]:
nodes = mappy.node
edges = {k:[] for k in tabu.bus}
for k in tabu.bus:
    for i in tabu.best[k]:
        n = tabu.best[k][i]
        if i != tabu.model.last:
            edges[k].append((i,n.next.key))
objtabu = tabu.best[-2]
displaygraph(nodes, edges,'Tabu')    
nodes,edges, tabu.best

In [ ]:
# tabu.best[0] = LinkedList()
# [0,1,4,2,6,7,9,11]
for i in tabu.bestcandidate[0]:
    print(tabu.bestcandidate[0][i].bus)

In [ ]:
tabutsp, t5 = Mod5()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in tabu.bus}
for k in tabutsp.bus:
    for i in tabutsp.best[k]:
        n = tabutsp.best[k][i]
        if i != tabutsp.model.last:
            edges[k].append((i,n.next.key))
objtabutsp = tabutsp.best[-2]
displaygraph(nodes, edges,'TabuTSP')    
nodes,edges, tabutsp.best

In [ ]:
t3,t4

In [ ]:
mappy.pickup_time

In [ ]:
# print(twosp.sim.alpha)
lshaped.printsol(lshaped.submodel)

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(lshaped.bus)}
for i, j in lshaped.parameters.edges:
    for k in range(lshaped.bus):
        if lshaped.submodel[3].variables.xs[i,j,k].X > 0.5:
            if i == 0 and j == 9:
                continue
            if j != lshaped.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))
bounds = lshaped.getcancel()
objlshaped = lshaped.model.ObjVal
displaygraph(nodes, edges,'L-Shaped')
nodes,edges

In [ ]:
drpstw